In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [24]:
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.io as sio
import warnings

warnings.filterwarnings('always')

def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    print(newX.shape)
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels



def split_balanced(data, target, test_size=0.5):

    classes = np.unique(target)
    # can give test_size as fraction of input data size of number of samples
    if test_size<1:
        n_test = np.round(len(target)*test_size)
    else:
        n_test = test_size
    n_train = max(0,len(target)-n_test)
    n_train_per_class = max(1,int(np.floor(n_train/len(classes))))
    n_test_per_class = max(1,int(np.floor(n_test/len(classes))))

    ixs = []
    for cl in classes:
        if (n_train_per_class+n_test_per_class) > np.sum(target==cl):
            # if data has too few samples for this class, do upsampling
            # split the data to training and testing before sampling so data points won't be
            #  shared among training and test data
            splitix = int(np.ceil(n_train_per_class/(n_train_per_class+n_test_per_class)*np.sum(target==cl)))
            ixs.append(np.r_[np.random.choice(np.nonzero(target==cl)[0][:splitix], n_train_per_class),
                np.random.choice(np.nonzero(target==cl)[0][splitix:], n_test_per_class)])
        else:
            ixs.append(np.random.choice(np.nonzero(target==cl)[0], n_train_per_class+n_test_per_class,
                replace=False))

    # take same num of samples from all classes
    ix_train = np.concatenate([x[:n_train_per_class] for x in ixs])
    ix_test = np.concatenate([x[n_train_per_class:(n_train_per_class+n_test_per_class)] for x in ixs])

    X_train = data[ix_train,:]
    X_test = data[ix_test,:]
    y_train = target[ix_train]
    y_test = target[ix_test]

    return X_train, X_test, y_train, y_test




t = sio.loadmat('/content/gdrive/My Drive/indiana/Pavia/PaviaU.mat')
f = sio.loadmat('/content/gdrive/My Drive/indiana/Pavia/PaviaU_gt.mat')
X = t['paviaU']
y = f['paviaU_gt']
# print(X.shape)
# print(y.shape)
# print(np.unique(y))
windowSize = 25
K = 15
X,pca = applyPCA(X,numComponents=K)
Xtest, ytest = createImageCubes(X, y, windowSize=windowSize)
test_ratio = 0.98
x_train, x_test, y_train, y_test = splitTrainTestSet(Xtest, ytest, test_ratio)
# x_train, x_test, y_train, y_test = split_balanced(Xtest, ytest, test_ratio)

print(x_test.shape)
print(y_test.shape)
print(np.unique(y_test))
print('e')
print('x_train',x_train.shape)
print('y_train',y_train.shape)
print('unique(y_train)',np.unique(y_train))
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

(634, 364, 15)
(41921, 25, 25, 15)
(41921,)
[0. 1. 2. 3. 4. 5. 6. 7. 8.]
e
x_train (855, 25, 25, 15)
y_train (855,)
unique(y_train) [0. 1. 2. 3. 4. 5. 6. 7. 8.]


{0.0: 132,
 1.0: 373,
 2.0: 42,
 3.0: 61,
 4.0: 27,
 5.0: 100,
 6.0: 27,
 7.0: 74,
 8.0: 19}

In [25]:
np.save('/content/gdrive/My Drive/indiana/Pavia/'+'x_train_PaviaUni'+'test_ratio'+str(test_ratio)+'windowSize'+str(windowSize)+'pca'+str(K), x_train)

In [26]:
np.save('/content/gdrive/My Drive/indiana/Pavia/'+'x_test_PaviaUni'+'test_ratio'+str(test_ratio)+'windowSize'+str(windowSize)+'pca'+str(K), x_test)

In [27]:
np.save('/content/gdrive/My Drive/indiana/Pavia/'+'y_train_PaviaUni'+'test_ratio'+str(test_ratio)+'windowSize'+str(windowSize)+'pca'+str(K), y_train)

In [28]:
np.save('/content/gdrive/My Drive/indiana/Pavia/'+'y_test_PaviaUni'+'test_ratio'+str(test_ratio)+'windowSize'+str(windowSize)+'pca'+str(K), y_test)

In [ ]:
unique, counts = np.unique(ytest, return_counts=True)
dict(zip(unique, counts))

{0.0: 6631,
 1.0: 18649,
 2.0: 2099,
 3.0: 3064,
 4.0: 1345,
 5.0: 5029,
 6.0: 1330,
 7.0: 3682,
 8.0: 947}